In [1]:
import pandas as pd
import numpy as np

In [10]:
original_data = pd.read_csv("Original Data/original_data.csv", low_memory=False)

In [11]:
# prune all variables with mv > 50%
original_data.describe()

,login,grp,xsect,dem02,dem03,dem04,dem05,dem05_32,dem05_16,dem05_8,...,nvalid_ph,ph_t,nvalid_mh,mh_t,sgrq_sx_r,sgrq_act_r,sgrq_imp_r,sgrq_tot_r,Unnamed: 949,Unnamed: 950
count,2796.000000,2147.000000,2793.000000,2796.000000,2796.000000,2777.000000,2750.000000,2729.000000,2750.0,2750.000000,...,341.000000,336.000000,336.000000,336.000000,3.340000e+02,3.320000e+02,3.310000e+02,3.310000e+02,9.000000e+00,1.000000e+00
mean,57436.652361,1.674429,1.625492,63.317597,1.442418,0.013684,1.622182,0.002565,0.0,0.029091,...,5.187683,350.059524,12.050595,392.160714,1.247146e+16,9.502412e+15,1.408852e+16,1.243517e+16,1.404604e+16,5.985361e+15
std,24407.716248,0.468697,0.484082,9.566788,0.496762,0.116196,2.303022,0.050591,0.0,0.168092,...,20.420289,133.354010,55.637423,164.689118,1.554408e+16,1.423946e+16,1.580091e+16,1.491821e+16,2.121146e+16,NaN
min,10341.000000,1.000000,1.000000,41.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,...,3.000000,4.000000,2.000000,4.000000,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+01,5.312472e+14,5.985361e+15
25%,38692.000000,1.000000,1.000000,55.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,...,4.000000,308.000000,4.000000,347.250000,4.865522e+15,8.451879e+14,4.448484e+15,4.784877e+15,7.677846e+14,5.985361e+15
50%,55474.000000,2.000000,2.000000,62.000000,1.000000,0.000000,1.000000,0.000000,0.0,0.000000,...,4.000000,369.000000,4.000000,426.000000,6.921509e+15,6.630552e+15,6.676740e+15,6.652819e+15,4.704467e+15,5.985361e+15
75%,78680.000000,2.000000,2.000000,71.000000,2.000000,0.000000,2.000000,0.000000,0.0,0.000000,...,4.000000,421.000000,4.000000,502.500000,9.280755e+15,8.526999e+15,1.889807e+16,8.677027e+15,7.276533e+15,5.985361e+15
max,97908.000000,2.000000,2.000000,88.000000,2.000000,1.000000,32.000000,1.000000,0.0,1.000000,...,381.000000,677.000000,482.000000,676.000000,6.336604e+16,6.176495e+16,6.352347e+16,7.156961e+16,5.141577e+16,5.985361e+15


In [ ]:
# Perform per-patient missing value imputation
from pandas import DataFrame
from pandas import concat
# first row are the variable names
# second row are the variable types
data = pd.read_excel("working_sheet.xlsx", sheet_name="variable_types", header=0)

var_names = data.columns
# get the second row
var_types = data.iloc[0, :]
# print(data.shape)
# drop the second row
data = data.drop(0, axis=0)
# print(data.shape)

numeric_vars = var_names[var_types == 'Numeric'].tolist()
categorical_vars = var_names[var_types == 'Categorical'].tolist()


# iterate through all unique values in the login column. Skip the second header row
for login in data.loc[:,"login"].unique():
    # get a selection of all rows with that login value
    login_selection = data[data["login"] == login]
    # iterate through each column of the selection
    for col in login_selection.columns:
        # check if there is at least one value in the selection
        if login_selection[col].count() > 0:
            # if the column is numeric
            if col in numeric_vars:
                print("converting numeric: "+str(col))
                # replace all missing values with the mean of the column
                login_selection[col] = login_selection[col].fillna(login_selection[col].median())
            # if the column is categorical
            elif col in categorical_vars:
                print("converting categorical: "+str(col))
                # replace all missing values with the mode of the column
                login_selection[col] = login_selection[col].fillna(login_selection[col].mode()[0])
    # replace the original data with the imputed data
    data[data["login"] == login] = login_selection
# write to a new file
data.to_csv("per_patient_imputation.csv", sep=";")


In [10]:
data = pd.read_csv("per_patient_imputation.csv", sep=";")
print(data.shape)
# drop all variables with mv > 60%
data = data.dropna(axis=1, thresh=0.4*data.shape[0])
print(data.shape)
# save data to file
data.to_csv("dropped_variables.csv", sep=";")

(2796, 910)
(2796, 524)


In [ ]:
data = pd.read_csv("dropped_variables.csv", sep=";")
# drop all patients with mv > 60%

In [8]:
import pandas_profiling
data = pd.read_csv("per_patient_imputation.csv", sep=";")

# count all cells that contain ''
# print(data[data == ''].count().sum() / data.size)



print(data.isna().sum().sum() / data.size)

# report = pandas_profiling.ProfileReport(data)
# report.to_file("report_cleaning_1.html")


0.417124149098398


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13432\1429530462.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling
